In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train = np.load('./data/train.npz')
val = np.load('./data/validation.npz')
test = np.load('./data/test.npz')

#### Train-test-split in training, validation and test data

In [ ]:
X_train, y_train = train['images'], train['labels']
X_val, y_val = val['images'], val['labels']
X_test, y_test = test['images'], test['labels']

## Model Creation

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding='same', input_shape = (48, 48, 1)))
model.add(Dropout(rate = 0.4))
model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides = (1, 1), padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
model.add(BatchNormalization(axis = -1))
model.add(LeakyReLU(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), padding='same', input_shape = (48, 48, 1)))
model.add(Dropout(rate = 0.4))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides = (1, 1), padding='same'))
model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
model.add(BatchNormalization(axis = -1))
model.add(LeakyReLU(0.2))

model.add(Flatten())

model.add(Dense(units = 256))
model.add(Dropout(rate = 0.4))
model.add(LeakyReLU(0.2))

model.add(Dense(units = 512))
model.add(Dropout(rate = 0.4))
model.add(Activation('relu'))

model.add(Dense(units = 7, activation = 'softmax'))

In [ ]:
adam = Adam(lr = 1e-3, epsilon=1e-8, decay=1e-5)

model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size = 256, epochs = 10, validation_data = (X_val, y_val))

In [ ]:
model.evaluate(X_val, y_val, batch_size = 128)

In [ ]:
predictions = model.predict(X_test, batch_size = 128)

In [ ]:
predictions = np.argmax(predictions, axis = 1)
y_true = np.argmax(y_test, axis = 1)

In [ ]:
np.mean(predictions == y_true)

In [ ]:
model.save('./models/my_model.h5')

## With keras image generator

In [ ]:
gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

In [ ]:
gen.fit(X_train)

train_gen = gen.flow(X_train, y_train, batch_size=256)
val_gen = gen.flow(X_val, y_val, batch_size=256)

In [ ]:
model2 = Sequential.from_config(model.get_config())

adam = Adam(lr = 1e-3, epsilon=1e-8, decay=1e-5)

model2.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model2.fit_generator(train_gen, epochs = 100, validation_data = (X_val, y_val))

In [ ]:
model2.save('best_yet_600_n.h5')